# Mini-Tarea 1: Análisis de datos de preferencias musicales

La presente actividad debe ser realizada de forma individual. El formato de entregar es el archivo .ipynb con todas las celdas ejecutadas. Las secciones donde se planteen preguntas de forma explícita, deben ser respondida en celdas de texto, y no se aceptará solo el output de una celda de código como respuesta.

**Fecha de entrega:** martes 13 de octubre de 2020, 09:00 hrs.

**Nombre alumno:** Darwin Joan Pérez Avendaño.


# Apache Hive

## Instalación de ambiente

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving hadoop_colab_installer.py to hadoop_colab_installer.py
User uploaded file "hadoop_colab_installer.py" with length 13493 bytes


In [2]:
exec(open('hadoop_colab_installer.py').read())

Active services:
2080 NodeManager
1780 DataNode
1624 ResourceManager
2153 JobHistoryServer
2201 Jps
1709 NameNode



## Descarga de dataset

In [3]:
!wget -q --show-progress http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-1K.tar.gz
!tar xzf lastfm-dataset-1K.tar.gz

lastfm-dataset-1K.t 100%[===================>] 641.58M  1.69MB/s    in 7m 12s  


## Actividad 0

In [ ]:
!ls

In [4]:
!hdfs dfs -mkdir hdfs_user_dir
!hdfs dfs -mkdir hdfs_profile_dir

In [5]:
!hdfs dfs -put lastfm-dataset-1K/userid-profile.tsv hdfs_user_dir
!hdfs dfs -put lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv hdfs_profile_dir

In [6]:
!hdfs dfs -ls  hdfs_user_dir
!hdfs dfs -ls  hdfs_profile_dir

Found 1 items
-rw-r--r--   1 root supergroup      37842 2020-10-13 03:58 hdfs_user_dir/userid-profile.tsv
Found 1 items
-rw-r--r--   1 root supergroup 2529193595 2020-10-13 03:59 hdfs_profile_dir/userid-timestamp-artid-artname-traid-traname.tsv


## Carga de tablas en **HiveQl**



In [7]:
%%writefile create_tables_usuario_profile.sql
CREATE TABLE usuario (userid STRING, gender STRING, age INT, country STRING, signup STRING) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t';
LOAD DATA INPATH 'hdfs_user_dir/userid-profile.tsv' INTO table usuario;

CREATE TABLE profile (userid STRING, times_tamp STRING, musicbrainz_artist_id STRING, artist_name STRING, musicbrainz_track_id STRING, track_name STRING) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t';
LOAD DATA INPATH 'hdfs_profile_dir/userid-timestamp-artid-artname-traid-traname.tsv' INTO table profile;


Writing create_tables_usuario_profile.sql


In [8]:
!hive -f create_tables_usuario_profile.sql

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.7-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.0/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]

Logging initialized using configuration in jar:file:/content/apache-hive-2.3.7-bin/lib/hive-common-2.3.7.jar!/hive-log4j2.properties Async: true
OK
Time taken: 9.627 seconds
Loading data to table default.usuario
OK
Time taken: 1.529 seconds
OK
Time taken: 0.212 seconds
Loading data to table default.profile
OK
Time taken: 0.566 seconds


## Actividad 1

In [9]:
%%writefile artista_mas_popular.sql
SELECT artist_name, COUNT(musicbrainz_track_id) AS Conteo FROM profile 
GROUP BY artist_name
ORDER BY Conteo DESC
LIMIT 10;

Writing artista_mas_popular.sql


In [10]:
!hive -f artista_mas_popular.sql

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.7-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.0/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]

Logging initialized using configuration in jar:file:/content/apache-hive-2.3.7-bin/lib/hive-common-2.3.7.jar!/hive-log4j2.properties Async: true
Query ID = root_20201013040102_a1180c69-19c4-495a-94c0-e420bcd83f1d
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 10
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.

Responda la siguiente pregunta:

¿ Cuál es el artista más popular y el menos popular del ranking top-10 ?

***R.*** El artista más popular del **Top-10** es **Radiohead** y el artista menos popular del **Top-10** es **Elliott Smith**.


## Actividad 2

In [11]:
%%writefile distribucion_por_genero.sql
SELECT gender, count(musicbrainz_track_id) AS Conteo FROM profile INNER JOIN usuario ON usuario.userid = profile.userid 
WHERE  length(gender) > 0 AND artist_name = 'Radiohead'
GROUP BY gender;

Writing distribucion_por_genero.sql


In [12]:
!hive -f distribucion_por_genero.sql

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.7-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.0/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]

Logging initialized using configuration in jar:file:/content/apache-hive-2.3.7-bin/lib/hive-common-2.3.7.jar!/hive-log4j2.properties Async: true
Query ID = root_20201013040856_565ad8eb-4151-4079-986e-945e47ab1696
Total jobs = 1
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.7-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.0/share/hadoop/common/lib/sl

Responda la siguiente pregunta:

¿ Cuál es la cantidad de hombres y mujeres que han escuchado alguna canción del artista más popular ?

***R.*** Un total de **43748 Mujeres** y un total de **63784 Hombres** han escuchado alguna canción de **Radiohead**.


## Actividad 3

In [13]:
%%writefile distribucion_por_edad.sql
SELECT age, count(musicbrainz_track_id) AS Conteo FROM profile INNER JOIN usuario ON usuario.userid = profile.userid 
WHERE age > 0 AND artist_name = 'Radiohead' AND age < 36
GROUP BY age
ORDER BY age ASC;

Writing distribucion_por_edad.sql


In [14]:
!hive -f distribucion_por_edad.sql

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.7-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.0/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]

Logging initialized using configuration in jar:file:/content/apache-hive-2.3.7-bin/lib/hive-common-2.3.7.jar!/hive-log4j2.properties Async: true
Query ID = root_20201013041138_da87711f-2ef6-4d14-a7f1-5e7ee760e3b1
Total jobs = 2
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.7-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.0/share/hadoop/common/lib/sl

Responda la siguiente pregunta:

¿ Cuántos usuarios de 35 años han escuchado al artista más popular ?

***R.*** **1543** usuarios de una edad de 35 años han escuchado **Radiohead**.